In [1]:
from scripts.DataHandler import DataHandler

handler = DataHandler(["Crime"])
ds = handler[0]

df = ds.fetch_data_frame()   # 不加 where，直接加载
print(df.head(20))
print(df.columns)
print(df.shape)

   Seven Major Felony Offenses  \
0                          NaN   
1                          PCT   
2                            1   
3                          NaN   
4                          NaN   
5                          NaN   
6                          NaN   
7                          NaN   
8                          NaN   
9                          NaN   
10                           5   
11                         NaN   
12                         NaN   
13                         NaN   
14                         NaN   
15                         NaN   
16                         NaN   
17                         NaN   
18                           6   
19                         NaN   

                                           Unnamed: 1  Unnamed: 2  Unnamed: 3  \
0                                                 NaN         NaN         NaN   
1                                               CRIME      2000.0      2001.0   
2   MURDER & NON NEGL. MANSLAUGHTER       

In [17]:
import pandas as pd

def load_crime_clean(path):
    # 读取完整 Excel（不设 header）
    df_raw = pd.read_excel(path, header=None)

    # 找到 "STATISTICAL NOTES" 所在行
    note_rows = df_raw[df_raw.iloc[:, 0] == "STATISTICAL NOTES"].index
    if len(note_rows) > 0:
        cutoff = note_rows[0]
        df_raw = df_raw.iloc[:cutoff]  # 删掉 notes 后所有行

    # 第二行是 header
    header = df_raw.iloc[1]
    df = df_raw.iloc[2:].copy()
    df.columns = header

    # 清理 PCT / CRIME
    pct_col_name = df.columns[0]
    crime_col_name = df.columns[1]

    df["PCT"] = df[pct_col_name].ffill()
    df["CRIME"] = df[crime_col_name].ffill()

    # 找年份列
    year_cols = [c for c in df.columns if str(c).isdigit()]

    # 做 tidy (long-format)
    tidy = df.melt(
        id_vars=["PCT", "CRIME"],
        value_vars=year_cols,
        var_name="YEAR",
        value_name="COUNT"
    )

    # 类型转换
    tidy["PCT"] = tidy["PCT"].astype(str)
    tidy["YEAR"] = tidy["YEAR"].astype(int)
    tidy["COUNT"] = tidy["COUNT"].fillna(0).astype(int)

    return tidy

In [18]:
clean_df = load_and_clean_crime("/Users/davinkey/Desktop/capstone/seven-major-felony-offenses-by-precinct-2000-2024.xls")
print(clean_df.head())
print(clean_df.shape)

Empty DataFrame
Columns: [PCT, CRIME, YEAR, COUNT]
Index: []
(0, 4)


In [15]:
clean_df.head(5)

,PCT,CRIME,YEAR,COUNT


In [27]:
path = "/Users/davinkey/Desktop/capstone/seven-major-felony-offenses-by-precinct-2000-2024.xls"
df_raw = pd.read_excel(path, header=None)

# 找到 "STATISTICAL NOTES" 所在行
note_rows = df_raw[df_raw.iloc[:, 0] == "STATISTICAL NOTES"].index
print("Note rows:", note_rows)
if len(note_rows) > 0:
    cutoff = note_rows[0]
    df_raw = df_raw.iloc[:cutoff]  # 删掉 notes 后所有行

# 第二行是 header
header = df_raw.iloc[2]
df = df_raw.iloc[3:].copy()
df.columns = header
print("Columns after setting header:", df.columns)
print("Data shape after removing notes:", df.shape)


df["PCT"] = df["PCT"].ffill()
df = df[df["CRIME"].notna()]
print(df.tail(10))



Note rows: Index([638], dtype='int64')
Columns after setting header: Index([  'PCT', 'CRIME',  2000.0,  2001.0,  2002.0,  2003.0,  2004.0,  2005.0,
        2006.0,  2007.0,  2008.0,  2009.0,  2010.0,  2011.0,  2012.0,  2013.0,
        2014.0,  2015.0,  2016.0,  2017.0,  2018.0,  2019.0,  2020.0,  2021.0,
        2022.0,  2023.0,  2024.0],
      dtype='object', name=2)
Data shape after removing notes: (635, 27)
2    PCT                                              CRIME  2000.0  2001.0  \
625  123  GRAND LARCENY OF MOTOR VEHICLE                ...   266.0   193.0   
626  123                  TOTAL SEVEN MAJOR FELONY OFFENSES   636.0   514.0   
627  DOC  MURDER & NON NEGL. MANSLAUGHTER               ...     NaN     NaN   
628  DOC             RAPE                                       NaN     NaN   
629  DOC             ROBBERY                                    NaN     NaN   
630  DOC             FELONY ASSAULT                             NaN     NaN   
631  DOC             BURGLARY    

In [28]:
print(df["PCT"].dtype)

object


In [29]:
df["PCT"] = df["PCT"].astype("string")

In [31]:
df["PCT"] = df["PCT"].astype(str)

In [32]:
df.to_csv("felony_by_precinct_cleaned.csv", index=False)